## Access Summit EFD data from the LDF
---
[The EFD replication service](https://sqr-050.lsst.io/) runs at LDF and continuously pulls data from the Summit EFD

In [ ]:
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst_efd_client import EfdClient

Create the client pointed to the LDF EFD instance replpicated from the summit.  This will automatically retrieve credentials to use for access to the read only user for that particular instance

In [ ]:
efd_stable = EfdClient('ldf_stable_efd')

Now select some data.  This will take about 20 seconds to return ~1 million rows

In [ ]:
measured_fields = [f'measuredForce{el}' for el in range(6)]
end = Time('2022-03-01T07:00:00Z')
window = TimeDelta(60*60*6, format='sec')
actuators = await efd_stable.select_time_series('lsst.sal.MTM1M3.hardpointActuatorData', measured_fields+['forceMagnitude',], end-window, end)

It's nice having the full time resolution data, but for plotting over time spans of several hours, we can downsample.

In [ ]:
sample_window = '10min'
resampled_dict = {'forceMagnitude': actuators['forceMagnitude'].resample(sample_window).mean()}
for el in range(6):
    col = f'measuredForce{el}'
    resampled_dict[col] = actuators[col].resample(sample_window).mean()

actuators_resample = pd.DataFrame(resampled_dict, index=resampled_dict['forceMagnitude'].index)

Now plot the results.

In [ ]:
ax = actuators_resample.plot(figsize=(10,10))
_ = ax.set_xlabel('Time (UTC)')
_ = ax.set_ylabel('Force (N)')